In [8]:
import pickle
import numpy as np
import pandas as pd
import autosklearn
import matplotlib.pyplot as plt
import sys

In [9]:
births = pd.read_csv("../../birth_randoms/metrics_testset.csv").drop(["precision", "auroc", "lr_acc"], axis=1)
rings = pd.read_csv("../../ring_randoms/metrics_testset.csv").drop(["precision", "auroc", "lr_acc"], axis=1)
adults = pd.read_csv("../../adult_randoms/metrics_testset.csv").drop(["precision", "auroc", "lr_acc"], axis=1)
hearts = pd.read_csv("../../heart_randoms/metrics_testset.csv").drop(["precision", "auroc", "lr_acc"], axis=1)
hearts.head()

,algo,no,entropy,cm,dm,discern,ilm,hellinger,bivariate_corr,avg_eq_size,ambiguity,granularity,dse
0,datafly,1,0.918940,0.150711,0.062112,0.264964,0.062129,0.243611,0.496877,0.058824,0.031780,0.060316,0.284026
1,datafly_shuffled,1,0.918940,0.150711,0.062112,0.264964,0.062129,0.243611,0.496877,0.058824,0.031780,0.060316,0.284026
2,mondrian,1,0.623465,0.119803,0.919255,0.000680,0.058060,0.424854,0.029728,0.000653,0.000655,0.036652,0.274516
3,datafly,2,0.918940,0.150711,0.062112,0.264964,0.062129,0.243611,0.496877,0.058824,0.031780,0.060316,0.284026
4,datafly_shuffled,2,0.918940,0.150711,0.062112,0.264964,0.062129,0.243611,0.496877,0.058824,0.031780,0.060316,0.284026


In [10]:
births_accs = pd.read_csv("../../birth_randoms/accuracies_testset.csv")
rings_accs = pd.read_csv("../../ring_randoms/accuracies_testset.csv")
adults_accs = pd.read_csv("../../adult_randoms/accuracies_testset.csv")
hearts_accs = pd.read_csv("../../heart_randoms/accuracies_testset.csv")
hearts_accs.head()

,algo,no,knn_pca_auroc,knn_pca_acc,rf_pca_auroc,rf_pca_acc,lr_auroc,lr_acc,lr_pca_auroc,lr_pca_acc
0,datafly,1,0.591972,0.856557,0.667191,0.856557,0.67553,0.855191,0.671626,0.855191
1,datafly_shuffled,1,0.591972,0.856557,0.667510,0.856557,0.67553,0.855191,0.671626,0.855191
2,mondrian,1,0.599727,0.855191,0.689071,0.856557,0.53824,0.719945,0.670297,0.856557
3,datafly,2,0.591972,0.856557,0.669636,0.856557,0.67553,0.855191,0.671626,0.855191
4,datafly_shuffled,2,0.591972,0.856557,0.671611,0.856557,0.67553,0.855191,0.671626,0.855191


In [11]:
from random import randint

def gencoordinates(n, size):
    pairs = set()
    while len(pairs) < size:
        x, y = randint(1,n),randint(1,n)
        if (x,y) not in pairs and (y,x) not in pairs and x != y:
            pairs.add((x,y))
    return pairs
            
#gencoordinates(200)
larger_better = ["dm"]
def count_successes(df, accs, algo, target, no_trials, pairs, name):
    model = pickle.load(open(f"../../autosklearn_models/{name}_randoms_{target}_{algo}.pkl", "rb"))
    count = 0
    for (x,y) in pairs:
        x_met = (df[(df["no"] == x) & (df["algo"] == algo)]).drop(["algo", "no"], axis=1)
        y_met = (df[(df["no"] == y) & (df["algo"] == algo)]).drop(["algo", "no"], axis=1)
        x_acc = (accs[(accs["no"] == x) & (accs["algo"] == algo)][target])
        y_acc = (accs[(accs["no"] == y) & (accs["algo"] == algo)][target])
        
        x_pred = model.predict(x_met)
        y_pred = model.predict(y_met)
        x_acc = float(x_acc)
        y_acc = float(y_acc)
        if ((x_pred >= y_pred and x_acc >= y_acc) | (x_pred <= y_pred and x_acc <= y_acc)):
            count += 1
        
    return max(count, no_trials-count)
    

In [27]:

pairs = [(x,y) for x in range(1,21) for y in range(x+1,21)]
no_trials =  len(pairs)
results = {}
for (df,accs, name) in [(births, births_accs, "birth"), (rings, rings_accs, "ring"), 
                  (adults, adults_accs, "adult"), (hearts, hearts_accs, "heart")]:
    print(name)
    res = {}
    for algo in ["mondrian", "datafly", "datafly_shuffled"]:
        print(algo)
        count_algo = 0
        for target in ["lr_acc", "lr_auroc", "rf_pca_acc", "rf_pca_auroc", "knn_pca_acc", "knn_pca_auroc"]:
            count_target = count_successes(df, accs, algo, target, no_trials, pairs, name)
            print(f"\t{target}: {count_target}")
            count_algo += count_target
                  
        
        perc = count_algo / (6*no_trials)
        res[algo] = perc
        print(f"{algo}: {count_algo}, {perc}")   
    results[name] = res

results = pd.DataFrame.from_dict(results, orient="index")
results

birth
mondrian


KeyboardInterrupt: 

In [49]:
pairs = [(x,y) for x in range(0,20) for y in range(x+1,20)]

def cross_algo(dataset, target):
    results = {}
    metrs, accs, name = dataset
    for algo in ["mondrian", "datafly", "datafly_shuffled"]:
        
        res = {}
        model = pickle.load(open(f"../../autosklearn_models/{name}_randoms_{target}_{algo}.pkl", "rb"))

        for algo2 in ["mondrian", "datafly", "datafly_shuffled"]:
            if algo2==algo:
                res[algo2]="-"
            else:
                accs_test = accs[accs["algo"] == algo2].drop(["algo","no"],axis=1).reset_index(drop=True)
                metrs_test = metrs[metrs["algo"] == algo2].drop(["algo","no"],axis=1).reset_index(drop=True)
                true_utils = accs_test[target]
                pred_utils = model.predict(metrs_test)
                count=0
                for x,y in pairs:
                    x_pred, y_pred = pred_utils[x], pred_utils[y]
                    x_acc, y_acc = true_utils[x], true_utils[y]
                    if ((x_pred >= y_pred and x_acc >= y_acc) | (x_pred <= y_pred and x_acc <= y_acc)):
                        count += 1
                perc = count/len(pairs)
                res[algo2] =perc
        results[algo] = res
        
    return results


algo_cross_list = []
for data in [(births, births_accs, "birth"), (rings, rings_accs, "ring"), 
                  (adults, adults_accs, "adult"), (hearts, hearts_accs, "heart")]: 

    for target in ["lr_acc", "lr_auroc", "rf_pca_acc", "rf_pca_auroc", "knn_pca_acc", "knn_pca_auroc"]:
        algo_cross_list.append(cross_algo(data, target))
        

In [50]:
algos = ["mondrian", "datafly", "datafly_shuffled"]

res = np.zeros((3,3))
for i,a in enumerate(algos):
    for j, a2 in enumerate(algos):
        if i != j:
            algo_pair_sc = sum([res[a][a2] for res in algo_cross_list]) / len(algo_cross_list)
            res[i][j] = algo_pair_sc
            
res
            
        
        

array([[0.        , 0.60592105, 0.60679825],
       [0.66798246, 0.        , 0.76513158],
       [0.63070175, 0.82280702, 0.        ]])

In [51]:
df = pd.DataFrame(res, columns=algos)
df.round(3).to_csv("1v1_metas_cross_algo.csv")

In [54]:
pairs = [(x,y) for x in range(0,20) for y in range(x+1,20)]

def cross_set(algo, target):
    results = {}
    for df, df_acc, name in [(births, births_accs, "birth"), (rings, rings_accs, "ring"), 
                  (adults, adults_accs, "adult"), (hearts, hearts_accs, "heart")]: 
        res = {}
        model = pickle.load(open(f"../../autosklearn_models/{name}_randoms_{target}_{algo}.pkl", "rb"))

        for df2, df_acc2, name2 in [(births, births_accs, "birth"), (rings, rings_accs, "ring"),(adults, adults_accs, "adult"), (hearts, hearts_accs, "heart")]: 
            
            if name==name2:
                res[name2]="-"
            else:
                accs_test = df_acc2[df_acc2["algo"] == algo].drop(["algo","no"],axis=1).reset_index(drop=True)
                metrs_test = df2[df2["algo"] == algo].drop(["algo","no"],axis=1).reset_index(drop=True)
                true_utils = accs_test[target]
                pred_utils = model.predict(metrs_test)
                count=0
                for x,y in pairs:
                    x_pred, y_pred = pred_utils[x], pred_utils[y]
                    x_acc, y_acc = true_utils[x], true_utils[y]
                    if ((x_pred >= y_pred and x_acc >= y_acc) | (x_pred <= y_pred and x_acc <= y_acc)):
                        count += 1
                perc = count/len(pairs)
                res[name2] =perc
        results[name] = res
        
    return results

dataset_cross_list = []
for algo in algos: 
    for target in ["lr_acc", "lr_auroc", "rf_pca_acc", "rf_pca_auroc", "knn_pca_acc", "knn_pca_auroc"]:
        dataset_cross_list.append(cross_set(algo, target))
        
dataset_cross_list
        

[{'birth': {'birth': '-',
   'ring': 0.37894736842105264,
   'adult': 0.4421052631578947,
   'heart': 0.6210526315789474},
  'ring': {'birth': 0.4842105263157895,
   'ring': '-',
   'adult': 0.6210526315789474,
   'heart': 0.3736842105263158},
  'adult': {'birth': 0.4473684210526316,
   'ring': 0.6789473684210526,
   'adult': '-',
   'heart': 0.3631578947368421},
  'heart': {'birth': 0.46842105263157896,
   'ring': 0.5052631578947369,
   'adult': 0.4631578947368421,
   'heart': '-'}},
 {'birth': {'birth': '-',
   'ring': 0.5052631578947369,
   'adult': 0.4789473684210526,
   'heart': 0.5789473684210527},
  'ring': {'birth': 0.4263157894736842,
   'ring': '-',
   'adult': 0.4842105263157895,
   'heart': 0.5736842105263158},
  'adult': {'birth': 0.7157894736842105,
   'ring': 0.5736842105263158,
   'adult': '-',
   'heart': 0.45263157894736844},
  'heart': {'birth': 0.5157894736842106,
   'ring': 0.6631578947368421,
   'adult': 0.4421052631578947,
   'heart': '-'}},
 {'birth': {'birth': 

In [55]:
datasets = ["birth", "ring", "adult", "heart"]

res = np.zeros((4,4))
for i,a in enumerate(datasets):
    for j, a2 in enumerate(datasets):
        if i != j:
            ds_pair_sc = sum([res[a][a2] for res in dataset_cross_list]) / len(dataset_cross_list)
            res[i][j] = ds_pair_sc
            
res
     

array([[0.        , 0.56695906, 0.58304094, 0.71929825],
       [0.69649123, 0.        , 0.64093567, 0.78128655],
       [0.58538012, 0.62719298, 0.        , 0.75      ],
       [0.72339181, 0.67105263, 0.7502924 , 0.        ]])

In [56]:
df = pd.DataFrame(res, columns=datasets)
df.round(3).to_csv("1v1_metas_cross_sets.csv")

In [57]:
pairs = [(x,y) for x in range(0,20) for y in range(x+1,20)]

def cross_measure(algo, dataset):
    metrs, accs, name = dataset
    results = {}
    
    for target in ["lr_acc", "lr_auroc", "rf_pca_acc", "rf_pca_auroc", "knn_pca_acc", "knn_pca_auroc"]:
        res = {}
        model = pickle.load(open(f"../../autosklearn_models/{name}_randoms_{target}_{algo}.pkl", "rb"))

        for target2 in ["lr_acc", "lr_auroc", "rf_pca_acc", "rf_pca_auroc", "knn_pca_acc", "knn_pca_auroc"]:
            
            if target==target2:
                res[target2]="-"
            else:
                accs_test = accs[accs["algo"] == algo].drop(["algo","no"],axis=1).reset_index(drop=True)
                metrs_test = metrs[metrs["algo"] == algo].drop(["algo","no"],axis=1).reset_index(drop=True)
                true_utils = accs_test[target2]
                pred_utils = model.predict(metrs_test)
                count=0
                for x,y in pairs:
                    x_pred, y_pred = pred_utils[x], pred_utils[y]
                    x_acc, y_acc = true_utils[x], true_utils[y]
                    if ((x_pred >= y_pred and x_acc >= y_acc) | (x_pred <= y_pred and x_acc <= y_acc)):
                        count += 1
                perc = count/len(pairs)
                res[target2] =perc
        results[target] = res
        
    return results

measure_cross_list = []
for algo in algos: 
    for data in [(births, births_accs, "birth"), (rings, rings_accs, "ring"), 
                  (adults, adults_accs, "adult"), (hearts, hearts_accs, "heart")]: 
        measure_cross_list.append(cross_measure(algo, data))
        
measure_cross_list
        

[{'lr_acc': {'lr_acc': '-',
   'lr_auroc': 0.6684210526315789,
   'rf_pca_acc': 0.6105263157894737,
   'rf_pca_auroc': 0.45789473684210524,
   'knn_pca_acc': 0.6052631578947368,
   'knn_pca_auroc': 0.6263157894736842},
  'lr_auroc': {'lr_acc': 0.5052631578947369,
   'lr_auroc': '-',
   'rf_pca_acc': 0.5210526315789473,
   'rf_pca_auroc': 0.5,
   'knn_pca_acc': 0.6105263157894737,
   'knn_pca_auroc': 0.6894736842105263},
  'rf_pca_acc': {'lr_acc': 0.6,
   'lr_auroc': 0.6157894736842106,
   'rf_pca_acc': '-',
   'rf_pca_auroc': 0.6052631578947368,
   'knn_pca_acc': 0.5947368421052631,
   'knn_pca_auroc': 0.5105263157894737},
  'rf_pca_auroc': {'lr_acc': 0.6052631578947368,
   'lr_auroc': 0.7105263157894737,
   'rf_pca_acc': 0.5842105263157895,
   'rf_pca_auroc': '-',
   'knn_pca_acc': 0.6947368421052632,
   'knn_pca_auroc': 0.5842105263157895},
  'knn_pca_acc': {'lr_acc': 0.631578947368421,
   'lr_auroc': 0.6631578947368421,
   'rf_pca_acc': 0.5263157894736842,
   'rf_pca_auroc': 0.58947

In [59]:
targets = ["lr_acc", "lr_auroc", "rf_pca_acc", "rf_pca_auroc", "knn_pca_acc", "knn_pca_auroc"]

res = np.zeros((6,6))
for i,a in enumerate(targets):
    for j, a2 in enumerate(targets):
        if i != j:
            ds_pair_sc = sum([res[a][a2] for res in measure_cross_list]) / len(measure_cross_list)
            res[i][j] = ds_pair_sc
            
res
     

array([[0.        , 0.55350877, 0.72105263, 0.49254386, 0.67280702,
        0.62982456],
       [0.61052632, 0.        , 0.69605263, 0.69561404, 0.64166667,
        0.57982456],
       [0.63201754, 0.64649123, 0.        , 0.62675439, 0.65219298,
        0.55833333],
       [0.57631579, 0.72719298, 0.68464912, 0.        , 0.62236842,
        0.60701754],
       [0.70482456, 0.64780702, 0.70833333, 0.62324561, 0.        ,
        0.63815789],
       [0.64298246, 0.70131579, 0.72324561, 0.70087719, 0.64517544,
        0.        ]])

In [60]:
df = pd.DataFrame(res, columns=targets)
df.round(3).to_csv("1v1_metas_cross_measure.csv")